In [1]:
from eval_utils import *
from data.data_loader import *
import ast
import os
import copy
import matplotlib.pyplot as plt
from google import genai
from google.genai import types
import tqdm
import json
import time
import pandas as pd
import numpy as np

PROJECT_ROOT = "/Users/satoshinakamoto/Documents/education/mcgill_cs_2023/2025_FALL/comp545/project/code/visual-cot"
os.chdir(PROJECT_ROOT)

/Users/satoshinakamoto/Documents/education/mcgill_cs_2023/2025_FALL/comp545/project/code/visual-cot/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## SETTING MODEL & API

In [ ]:
# Setting API
client = genai.Client(api_key=key)

# Setting model
MODEL = "gemini-2.5-pro"

## HELPER FUNCTIONS

In [ ]:
# Helper functions

def board_to_prompt(board):
    # Format the board into string representation
    formatted = []
    for row in board:
        new_row = []
        for cell in row:
            if cell is None:
                new_row.append('.')
            else:
                new_row.append(str(cell))
        formatted.append(new_row)

    # Create compact single-quote format for LLM prompt
    board_str = "[ " + ", ".join(
        ["['" + "','".join(row) + "']" for row in formatted]
    ) + " ]"

    return board_str




def create_prompt(exit, board, fsp_examples):

    prompt = f'''You have to solve a 6x6 rush hour puzzle.
Your goal is to move the Red car out. 
On the board, 'R' designates the Red car. 
The exit is located at {exit}.
Here are some few examples. 

Example 1) 
Input: {fsp_examples[0][0]}
Output: {fsp_examples[0][1]}

Example 2) 
Input: {fsp_examples[1][0]}
Output: {fsp_examples[1][1]}

Example 3) 
Input: {fsp_examples[2][0]}
Output: {fsp_examples[2][1]}
                
The following puzzle is the one you have to solve:
Input: {board}
Remember that the exit is located at {exit} and that you have to move the 'R' car.
Provide only the text response with no bolding or formatting and do not include the word "Output:"'''
    
    return prompt




def create_request(requests_path, pid, prompt_text, model=MODEL, effort="medium"):

    req = {
        "custom_id": str(pid),
        "method": "POST",
        "url": "/v1/responses",
        "body": {
            "model": model,
            "reasoning": { "effort": effort},
            "input": [
                {"role": "user", "content": prompt_text}
            ],
        }
    }

    with open(requests_path, "a") as f:
        f.write(json.dumps(req) + "\n")






def create_request_path(batch_number):
    return f"./eval/results/gpt/few_shots/requests_batch_{batch_number}.jsonl"




def check_batch_status(batch_id):
    print(f"Polling status for job: {batch_id}")

    while True:
        job = client.batches.retrieve(batch_id)
        print("Current status:", job.status)

        if job.status in ["completed", "failed", "expired", "cancelled"]:
            break
        
        time.sleep(60)

    print("Job finished with status:", job.status)





def write_batch_output(batch_id, responses_path, batch_number = 1):

    print(f"Writing output of batch ID: {batch_id}")

    batch = client.batches.retrieve(batch_id)

    print("Batch state:", batch.status)

    if batch.status == "completed":

        output_file_id = batch.output_file_id
        print("Output file ID:", output_file_id)

        file_bytes = client.files.content(output_file_id).read()

        with open(responses_path, "ab") as f:
            f.write(file_bytes)

        print(f"Saved batch output to responses_{batch_number}.jsonl")

    elif batch.status in ["failed", "expired"]:
        print(f"Batch {batch_number} with failed or expired.")
        print(batch)

    else:
        print("Batch not done yet:", batch.status)